The file test the trained MLP and LSTM in folder 'Building models' on all the data.   
Also, the last cell test the LSTM for the 20% unseen tracks data.

In [ ]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import datetime, time
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from livelossplot import PlotLosses
from pycm import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from torchsummary import summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# to import utilitis
import sys
sys.path.insert(0,'/content/drive/MyDrive/IRP/codefinal')
from utilities import convert_deg_to_sin_cos,normalize,MLPDataset,simpleFFN,mlptrain,mlpvalidate,generate_random_numbers

In [ ]:
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


In [ ]:
#prepare the data
m25 = np.load('/content/drive/MyDrive/IRP/data/structured/DepthContour25m.npy')
m25 = np.delete(m25, (45,83,77),axis=0)
print(f"Before reshaping: {np.shape(m25)}")


m0 = np.load('/content/drive/MyDrive/IRP/data/structured/Coastline0p01.npy')
m0 = np.delete(m0, (45,83,77),axis=0)
print(f"Before reshaping: {np.shape(m0)}")

# for each trach, each parameter, the first timestep data is just the index of the stations
m25 = m25[:,1:,:,:]
m0 = m0[:,1:,:,:]

#tar is SWH at coastline
tar = m0[:,:,:,2]
print(tar.shape)

tar[np.isnan(tar)] = 0
tar_rs = np.reshape(tar, (np.shape(tar)[0]*np.shape(tar)[1],np.shape(tar)[2]))

feature25m = np.reshape(m25, (np.shape(m25)[0]*np.shape(m25)[1],np.shape(m25)[2],np.shape(m25)[3]))
print(f"After reshaping: {feature25m.shape}")
feature25m = convert_deg_to_sin_cos(feature25m)
print(feature25m.shape)

#further reshape
feature25m = feature25m.reshape(feature25m.shape[0],feature25m.shape[1]*feature25m.shape[2])
print(feature25m.shape)

mean25,std25,feature25m = normalize(feature25m)



Before reshaping: (185, 98, 155, 6)
Before reshaping: (185, 98, 145, 6)
(185, 97, 145)
After reshaping: (17945, 155, 6)
(17945, 155, 8)
(17945, 1240)


In [ ]:
#test on lstm, model prediction is from a sequence length of 8 to the next one.
lstm = torch.load('/content/drive/MyDrive/IRP/model/LSTM25m_structured.npy')
data = MyDataset(feature25m,tar_rs,sequence_length=9)
dataloader = DataLoader(data, batch_size=len(data), shuffle=False)
allfeature, alltar = next(iter(dataloader))
allres = lstm(allfeature.to(device))

alltar = alltar[:,-1,:].cpu().detach().numpy().flatten()
allres = allres[:,-1,:].cpu().detach().numpy().flatten()

calculate_metrics(alltar, allres)

{'Bias': -0.0022535980850514185,
 'RMSE': 0.036833776821499895,
 'Scatter Index': 0.22309196082752314,
 'Correlation Coefficient': 0.9958883818307639,
 'Coefficient of Efficiency': 0.9915870544306082,
 'Index of Agreement': 0.9978556244334832,
 'MAE': 0.019833476474851237}

In [ ]:
# test the mlp
mlp = torch.load('/content/drive/MyDrive/IRP/model/MLP25m3.npy')

res = mlp(torch.tensor(feature25m).float().to(device))
res = res.cpu().detach().numpy()

res = res.flatten()
tar = tar_rs.flatten()

calculate_metrics(tar, res)

{'Bias': -0.00283972562254786,
 'RMSE': 0.026744819191328133,
 'Scatter Index': 0.16963800980323387,
 'Correlation Coefficient': 0.9976648499609193,
 'Coefficient of Efficiency': 0.9952812841038714,
 'Index of Agreement': 0.9988172166400493,
 'MAE': 0.011501265629603265}

In [ ]:
#LSTM on unseen tracks
split = int((len(tar_rs)*0.8))
X_test,y_test = feature25m[split:,:], tar_rs[split:,:]

data = MyDataset(X_test,y_test,sequence_length=9)
dataloader = DataLoader(data, batch_size=len(data), shuffle=False)
allfeature, alltar = next(iter(dataloader))
allres = lstm(allfeature.to(device))

alltar = alltar[:,-1,:].cpu().detach().numpy().flatten()
allres = allres[:,-1,:].cpu().detach().numpy().flatten()

calculate_metrics(alltar, allres)

{'Bias': -0.0033361478404443762,
 'RMSE': 0.03180298774257002,
 'Scatter Index': 0.1844140832036965,
 'Correlation Coefficient': 0.9972682560945663,
 'Coefficient of Efficiency': 0.9941746375554316,
 'Index of Agreement': 0.9985118583437433,
 'MAE': 0.017831796780339873}